# Setup

In [ ]:
import ammonite.ops._
import ammonite.ops.ImplicitWd._

In [ ]:
// This folder is used to store generated REPL classes, which will later be used in UDFs.
// Please provide an empty folder path. This is essential for Snowpark UDFs to work
val replClassPath = pwd+"/repl_classes"

// Delete any old files in the directory.
import sys.process._
s"rm -rf $replClassPath" !

// Create the REPL class folder.
import sys.process._
s"mkdir -p $replClassPath" !

In [ ]:
import $ivy.`com.snowflake:snowpark:1.4.0`
import com.snowflake.snowpark._
import com.snowflake.snowpark.functions._

Add the Ammonite kernel classes as dependencies for your UDF.

In [ ]:
def finalize_session_setup(session: Session): Unit = {
    def addClass(session: Session, className: String): String = {
      var cls1 = Class.forName(className)
      val resourceName = "/" + cls1.getName().replace(".", "/") + ".class"
      val url = cls1.getResource(resourceName)
      val path = url.getPath().split(":").last.split("!").head
      session.addDependency(path)
      path
    }
    addClass(session, "ammonite.repl.ReplBridge$")
    addClass(session, "ammonite.interp.api.APIHolder")
    addClass(session, "pprint.TPrintColors")
}

# Code

In [ ]:
val session = Session.builder.configs(Map(
    "URL" -> ("https://" + sys.env.get("SNOWFLAKE_DEFAULT_ORG").get + ".snowflakecomputing.com"),
    "USER" -> sys.env.get("SNOWFLAKE_USER").get,
    "PASSWORD" -> sys.env.get("SNOWFLAKE_PASSWORD").get,
    "ROLE" -> sys.env.get("SNOWFLAKE_ROLE").get,
    "WAREHOUSE" -> sys.env.get("SNOWFLAKE_DEFAULT_WAREHOUSE").get,
    "DB" -> sys.env.get("SNOWFLAKE_DEFAULT_DB").get,
    "SCHEMA" -> sys.env.get("SNOWFLAKE_DEFAULT_SCHEMA").get
)).create
finalize_session_setup(session)

In [ ]:
session.sql("show tables").show

In [ ]:
session.table("flights").show(5)

In [ ]:
session.table("flights").select(col("origin"), col("dest"), col("dep_time"), col("arr_time"), (col("dep_time") * 2).alias("double_dep_time")).show